In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
# Function to calculate return MAC from each layer

def mac_each_layer(l):
    o_shape, i_shape, ks, filters = ['', '', ''], ['', '', ''], [0, 0], [0, 0]
    macc = 0
    if ('Dense' in str(l)):
        print(l.input_shape)
        print(l.output_shape)

        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        print(i_shape)
        print(o_shape)
        macc = (o_shape[0] * i_shape)

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]
        macc = (filters * o_shape[0] * o_shape[1]) * (i_shape[2] * ks[0] * ks[1] )
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = (o_shape[0] *o_shape[1]) * (i_shape[2] * ks[0] * ks[1] )

    return macc

# Function to pass layers to the calculating function and to get the macc of each layer

def MAC_summary(model):
  print('%25s | %6s' % ('Layer Name', 'MACs'))
  print('*' * 100)
  t_macc = 0
  macc_dict = {}
  prefix_name = model.name + "_"
  for l in model.layers:
    if (type(l) in [models.Sequential, keras.Model]):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        macc_dict[prefix_name+l.name] = mac_obtained
        if ib_layer.name.endswith('conv'):
          print('%25s(%5s) | %5.4f' % (
                  prefix_name+l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained

    else:
      mac_obtained = mac_each_layer(l)
      macc_dict[prefix_name+l.name] = mac_obtained
      if l.name.endswith('conv'):
        print('%25s | %5.4f' % (
                  prefix_name+l.name, mac_obtained ))

    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return macc_dict

In [3]:
# Obtains the output in bits, the memory requirements of each layer

def keras_model_output(model):
    print('%25s | %6s' % ('Layer Name', 'Output bits'))
    print('*' * 100)
    output_dict = {}
    default_dtype = tf.keras.backend.floatx()
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_output(layer)
        else:
            no_of_neurons = tf.as_dtype(layer.dtype or default_dtype).size # dtype is float 32 and the size here is 4 bytes(32 bits)
            out_shape = layer.output_shape
            if isinstance(out_shape, list):
                for s in out_shape[0]:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes
            else:
                for s in out_shape:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes
            no_of_neurons *= 8 # Output obtained in bits
        output_dict[prefix_name + layer.name] = no_of_neurons
        print('%25s | %5.4f' % (
                  prefix_name+layer.name, no_of_neurons ))


    return output_dict

In [4]:
# EFFECTIVE DATA COMMUNICATION PROBABILITY

# Function to print the effective data communicated
def data_summary(model,x_test,y_test):

  # Creating dictionary for storing the percent of data communicated eliminating the sparse data
  effective_probability = {}

  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"

    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)

    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)

  # Compiling the cloned model and take the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  # m2.fit(x_test, y_test, epochs=5)
  m2.evaluate(x_test[0:100], y_test[0:100])

  print('%25s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  print("*" * 150)
  for l in m2.metrics:
    # print('%40s | %5.4f' % (l.name, l.result().numpy()))
    if l.name!="loss" and l.name!="accuracy" :
      effective_probability[l.name] = l.result().numpy()
      print('%25s | %5.4f' % (
                  l.name, effective_probability[l.name] ))

  return effective_probability


In [5]:
# ImageNet classification dataset for Resnet18 and ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

169001437/169001437 [==============================] - 2s 0us/step


In [6]:
# Calculations for Resnet50 CNN model
from tensorflow.keras.applications.resnet50 import ResNet50
model_Resnet50 = ResNet50(include_top=False,
                        weights='imagenet',
                        input_shape=(32, 32, 3),
                        pooling=None)

94765736/94765736 [==============================] - 1s 0us/step


In [7]:
MAC_summary(model_Resnet50)

               Layer Name |   MACs
****************************************************************************************************
      resnet50_conv1_conv | 2408448.0000
resnet50_conv2_block1_1_conv | 262144.0000
resnet50_conv2_block1_2_conv | 2359296.0000
resnet50_conv2_block1_0_conv | 1048576.0000
resnet50_conv2_block1_3_conv | 1048576.0000
resnet50_conv2_block2_1_conv | 1048576.0000
resnet50_conv2_block2_2_conv | 2359296.0000
resnet50_conv2_block2_3_conv | 1048576.0000
resnet50_conv2_block3_1_conv | 1048576.0000
resnet50_conv2_block3_2_conv | 2359296.0000
resnet50_conv2_block3_3_conv | 1048576.0000
resnet50_conv3_block1_1_conv | 524288.0000
resnet50_conv3_block1_2_conv | 2359296.0000
resnet50_conv3_block1_0_conv | 2097152.0000
resnet50_conv3_block1_3_conv | 1048576.0000
resnet50_conv3_block2_1_conv | 1048576.0000
resnet50_conv3_block2_2_conv | 2359296.0000
resnet50_conv3_block2_3_conv | 1048576.0000
resnet50_conv3_block3_1_conv | 1048576.0000
resnet50_conv3_block3_2_conv | 23

{'resnet50_input_1': 0,
 'resnet50_conv1_pad': 0,
 'resnet50_conv1_conv': 2408448,
 'resnet50_conv1_bn': 0,
 'resnet50_conv1_relu': 0,
 'resnet50_pool1_pad': 0,
 'resnet50_pool1_pool': 0,
 'resnet50_conv2_block1_1_conv': 262144,
 'resnet50_conv2_block1_1_bn': 0,
 'resnet50_conv2_block1_1_relu': 0,
 'resnet50_conv2_block1_2_conv': 2359296,
 'resnet50_conv2_block1_2_bn': 0,
 'resnet50_conv2_block1_2_relu': 0,
 'resnet50_conv2_block1_0_conv': 1048576,
 'resnet50_conv2_block1_3_conv': 1048576,
 'resnet50_conv2_block1_0_bn': 0,
 'resnet50_conv2_block1_3_bn': 0,
 'resnet50_conv2_block1_add': 0,
 'resnet50_conv2_block1_out': 0,
 'resnet50_conv2_block2_1_conv': 1048576,
 'resnet50_conv2_block2_1_bn': 0,
 'resnet50_conv2_block2_1_relu': 0,
 'resnet50_conv2_block2_2_conv': 2359296,
 'resnet50_conv2_block2_2_bn': 0,
 'resnet50_conv2_block2_2_relu': 0,
 'resnet50_conv2_block2_3_conv': 1048576,
 'resnet50_conv2_block2_3_bn': 0,
 'resnet50_conv2_block2_add': 0,
 'resnet50_conv2_block2_out': 0,
 'res

In [8]:
keras_model_output(model_Resnet50)

               Layer Name | Output bits
****************************************************************************************************
         resnet50_input_1 | 98304.0000
       resnet50_conv1_pad | 138624.0000
      resnet50_conv1_conv | 524288.0000
        resnet50_conv1_bn | 524288.0000
      resnet50_conv1_relu | 524288.0000
       resnet50_pool1_pad | 663552.0000
      resnet50_pool1_pool | 131072.0000
resnet50_conv2_block1_1_conv | 131072.0000
resnet50_conv2_block1_1_bn | 131072.0000
resnet50_conv2_block1_1_relu | 131072.0000
resnet50_conv2_block1_2_conv | 131072.0000
resnet50_conv2_block1_2_bn | 131072.0000
resnet50_conv2_block1_2_relu | 131072.0000
resnet50_conv2_block1_0_conv | 524288.0000
resnet50_conv2_block1_3_conv | 524288.0000
resnet50_conv2_block1_0_bn | 524288.0000
resnet50_conv2_block1_3_bn | 524288.0000
resnet50_conv2_block1_add | 524288.0000
resnet50_conv2_block1_out | 524288.0000
resnet50_conv2_block2_1_conv | 131072.0000
resnet50_conv2_block2_1_bn | 131072

{'resnet50_input_1': 98304,
 'resnet50_conv1_pad': 138624,
 'resnet50_conv1_conv': 524288,
 'resnet50_conv1_bn': 524288,
 'resnet50_conv1_relu': 524288,
 'resnet50_pool1_pad': 663552,
 'resnet50_pool1_pool': 131072,
 'resnet50_conv2_block1_1_conv': 131072,
 'resnet50_conv2_block1_1_bn': 131072,
 'resnet50_conv2_block1_1_relu': 131072,
 'resnet50_conv2_block1_2_conv': 131072,
 'resnet50_conv2_block1_2_bn': 131072,
 'resnet50_conv2_block1_2_relu': 131072,
 'resnet50_conv2_block1_0_conv': 524288,
 'resnet50_conv2_block1_3_conv': 524288,
 'resnet50_conv2_block1_0_bn': 524288,
 'resnet50_conv2_block1_3_bn': 524288,
 'resnet50_conv2_block1_add': 524288,
 'resnet50_conv2_block1_out': 524288,
 'resnet50_conv2_block2_1_conv': 131072,
 'resnet50_conv2_block2_1_bn': 131072,
 'resnet50_conv2_block2_1_relu': 131072,
 'resnet50_conv2_block2_2_conv': 131072,
 'resnet50_conv2_block2_2_bn': 131072,
 'resnet50_conv2_block2_2_relu': 131072,
 'resnet50_conv2_block2_3_conv': 524288,
 'resnet50_conv2_block2

In [9]:
data_summary(model_Resnet50,x_train, y_train)


4/4 [==============================] - 16s 194ms/step - loss: 10.3148 - accuracy: 0.0000e+00 - resnet50_input_1: 0.9591 - resnet50_conv1_pad: 0.6801 - resnet50_conv1_conv: 0.9900 - resnet50_conv1_bn: 0.9900 - resnet50_conv1_relu: 0.5291 - resnet50_pool1_pad: 0.4180 - resnet50_pool1_pool: 0.7122 - resnet50_conv2_block1_1_conv: 0.9980 - resnet50_conv2_block1_1_bn: 0.9980 - resnet50_conv2_block1_1_relu: 0.5116 - resnet50_conv2_block1_2_conv: 1.0000 - resnet50_conv2_block1_2_bn: 1.0000 - resnet50_conv2_block1_2_relu: 0.4683 - resnet50_conv2_block1_0_conv: 0.9980 - resnet50_conv2_block1_3_conv: 1.0000 - resnet50_conv2_block1_0_bn: 0.9980 - resnet50_conv2_block1_3_bn: 1.0000 - resnet50_conv2_block1_add: 1.0000 - resnet50_conv2_block1_out: 0.4623 - resnet50_conv2_block2_1_conv: 1.0000 - resnet50_conv2_block2_1_bn: 1.0000 - resnet50_conv2_block2_1_relu: 0.4412 - resnet50_conv2_block2_2_conv: 1.0000 - resnet50_conv2_block2_2_bn: 1.0000 - resnet50_conv2_block2_2_relu: 0.5050 - resnet50_conv2_blo

{'resnet50_input_1': 0.9591013590494791,
 'resnet50_conv1_pad': 0.6801383598799631,
 'resnet50_conv1_conv': 0.989959716796875,
 'resnet50_conv1_bn': 0.989959716796875,
 'resnet50_conv1_relu': 0.5290861129760742,
 'resnet50_pool1_pad': 0.4180433485243055,
 'resnet50_pool1_pool': 0.7122344970703125,
 'resnet50_conv2_block1_1_conv': 0.998046875,
 'resnet50_conv2_block1_1_bn': 0.998046875,
 'resnet50_conv2_block1_1_relu': 0.5116386413574219,
 'resnet50_conv2_block1_2_conv': 1.0,
 'resnet50_conv2_block1_2_bn': 1.0,
 'resnet50_conv2_block1_2_relu': 0.4683265686035156,
 'resnet50_conv2_block1_0_conv': 0.998046875,
 'resnet50_conv2_block1_3_conv': 1.0,
 'resnet50_conv2_block1_0_bn': 0.998046875,
 'resnet50_conv2_block1_3_bn': 1.0,
 'resnet50_conv2_block1_add': 1.0,
 'resnet50_conv2_block1_out': 0.46234893798828125,
 'resnet50_conv2_block2_1_conv': 1.0,
 'resnet50_conv2_block2_1_bn': 1.0,
 'resnet50_conv2_block2_1_relu': 0.44121551513671875,
 'resnet50_conv2_block2_2_conv': 1.0,
 'resnet50_conv